In [90]:
### Library Import Initialization

import numpy as np
import math
from math import *
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [91]:
### Function to Import Stock Tickers and Calculate Final Stock Price
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0


[*********************100%%**********************]  1 of 1 completed


152.25999450683594

In [92]:
### Sigma Calculation 
def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

get_sigma = compute_sigma(stock_data)
sigma = get_sigma.values[0]
sigma

0.32869474791095804

In [93]:
### Cumulative standard normal distribution
def cdf(x):
    return (1.0 + erf(x / np.sqrt(2.0))) / 2.0

### Compute phi Function
def phi(S_0, T, y, H, I, r, b, sigma):
    '''
    ϕ(S,T,y,H,I) represents the price of an American option with a continuous barrier. It incorporates the effects of early exercise 
    and the presence of a barrier, which can be either up-and-out, up-and-in, down-and-out, or down-and-in, depending on the 
    relative positions of the barrier level and the current asset price
    The function takes the following parameters:
    S_0: Current price of the underlying asset
    T: Time to expiration of the option
    y: The "nu" parameter, or gamma, which is often related to the cost of carry or dividend yield
    H: The barrier level
    I: The rebate, which is a cash payment given if the barrier is hit
    '''
    d1 = (np.log(S_0 / H) + (b + (y - 0.5) * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    phi = S_0 * cdf(d1) - np.exp(-r * T) * I * cdf(d2)
    
    return phi


In [94]:
# Compute psi Function
def psi(S_0, T, y, H, I_2, I_1, t_1, r, b, sigma):
    '''
    psi is a crucial part of the BS call option approximation formula as it incorporates the barrier feature and early exercise behavior into the option pricing calculation

    S_0: Current price of the underlying asset (stock price).
    T: Time to expiration of the option (in years).
    y: The "gamma" parameter, which may represent the cost of carry or dividend yield.
    H: The barrier level. This is the price level at which the barrier option comes into effect.
    I_2: Second flat boundary. This is related to the barrier structure and may have specific conditions for early exercise or option value adjustments.
    I_1​: First flat boundary. Similar to I_2​, this is also part of the barrier structure and affects the option's behavior.
    t_1​: Time boundary. This represents the time at which the second flat boundary (I_2​) comes into effect.
    r: Risk-free interest rate. This is typically the interest rate on a risk-free asset, such as a government bond.
    b: Cost of carry. This can include costs such as storage costs, financing costs, or dividend yield, depending on the context.
    sigma: Volatility of the underlying asset's returns. This measures the variability of the asset's price over time.
    '''
    psi = phi(S_0, t_1, y, H, I_2, r, b, sigma) - phi(S_0, t_1, y, H, I_1, r, b, sigma) \
          - I_2 * np.exp(-r * T) * (norm.cdf((-np.log(I_2 ** 2 / (S_0 * H)) + (b + (y - 0.5) * sigma ** 2) * T) / (sigma * np.sqrt(T)))
                                    - norm.cdf((-np.log(I_2 ** 2 / (S_0 * H))) / (sigma * np.sqrt(T)))) \
          + I_1 * np.exp(-r * T) * (norm.cdf((-np.log(I_1 ** 2 / (S_0 * H)) + (b + (y - 0.5) * sigma ** 2) * T) / (sigma * np.sqrt(T)))
                                    - norm.cdf((-np.log(I_1 ** 2 / (S_0 * H))) / (sigma * np.sqrt(T))))
    return psi

In [95]:
### Compute Bjerksund-Stensland Call
def bjerksund_stensland_call(S_0, K, T, r, b, sigma):
    '''
    S_0: Current price of the underlying asset (stock price).
    K: Asset strike price
    T: Time to expiration of the option (in years).
    H: The barrier level. This is the price level at which the barrier option comes into effect.
    I_2: Second flat boundary. This is related to the barrier structure and may have specific conditions for early exercise or option value adjustments.
    I_1​: First flat boundary. Similar to I_2​, this is also part of the barrier structure and affects the option's behavior.
    t_1​: Time boundary. This represents the time at which the second flat boundary (I_2​) comes into effect.
    r: Risk-free interest rate. This is typically the interest rate on a risk-free asset, such as a government bond.
    b: Cost of carry. This can include costs such as storage costs, financing costs, or dividend yield, depending on the context.
    B_0: The lower boundary or "floor" of the option's pricing domain. It is often associated with the intrinsic value of the option.
    B_infty: The upper boundary or "ceiling" of the option's pricing domain. It is the maximum value that the option can reach under certain conditions.
    sigma: Volatility of the underlying asset's returns. This measures the variability of the asset's price over time.
    '''
    # Calculate Beta, the exponent parameter in the Black-Scholes formula
    Beta = (0.5 - (b / sigma ** 2)) + np.sqrt(((b / sigma ** 2) - 0.5) ** 2 + 2 * r / sigma ** 2)

    # Calculate B_0 and B_infty, parameters related to the option's strike price and boundaries
    B_0 = Beta / (Beta - 1) * K
    B_infty = max(K, r / (r - b) * K)

    # Calculate t_1, a time parameter used in barrier option calculations
    t_1 = 0.5 * (np.sqrt(5) - 1) * T

    # Calculate h_1 and h_2, parameters related to the option's barrier and pricing
    h_1 = -((b * t_1) + 2 * sigma * np.sqrt(t_1)) * (K ** 2 / ((B_infty - B_0) * B_0))
    h_2 = -((b * T) + 2 * sigma * np.sqrt(T)) * (K ** 2 / ((B_infty - B_0) * B_0))

    # Calculate I_1 and I_2, trigger prices used in barrier option pricing
    I_1 = B_0 + (B_infty - B_0) * (1 - np.exp(h_1))
    I_2 = B_0 + (B_infty - B_0) * (1 - np.exp(h_2))

    # Calculate alpha_1 and alpha_2, coefficients used in option pricing formulas
    alpha_1 = (I_1 - K) * (I_1) ** (-Beta)
    alpha_2 = (I_2 - K) * (I_2) ** (-Beta)
    
    # Logic to determine whether to exercise option
    if (S_0 >= I_2).item() and b >= r:
        # If stock price >= upper trigger price (boundary has been met), option price = intrinsic value
        call_approx = S_0 - K
    else:
        # Compute the call option approximation using the BS formula and related functions
        call_approx = (
            alpha_2 * (S_0 ** Beta) - alpha_2 * phi(S_0, t_1, Beta, I_2, I_2, r, b, sigma) +
            phi(S_0, t_1, 1, I_2, I_2, r, b, sigma) - phi(S_0, t_1, 1, I_1, I_2, r, b, sigma) -
            K * phi(S_0, t_1, 0, I_2, I_2, r, b, sigma) + K * phi(S_0, t_1, 0, I_1, I_2, r, b, sigma) +
            alpha_1 * phi(S_0, t_1, Beta, I_1, I_2, r, b, sigma) - alpha_1 * psi(S_0, T, Beta, I_1, I_2, I_1, t_1, r, b,
                                                                                 sigma) +
            psi(S_0, T, 1, I_1, I_2, I_1, t_1, r, b, sigma) - psi(S_0, T, 1, K, I_2, I_1, t_1, r, b, sigma) -
            K * psi(S_0, T, 0, I_1, I_2, I_1, t_1, r, b, sigma) + psi(S_0, T, 0, K, I_2, I_1, t_1, r, b, sigma)
            )
    
    # in boundary conditions, this approximation can break down
    # Make sure option value is greater than or equal to European value
    #value = max(value, e_value)

    return t_1, h_1, h_2, I_1, I_2, call_approx

In [96]:
### Function Call
# Compute the Bjerksund-Stensland call approximation
t_1, h_1, h_2, I_1, I_2, call_approx = bjerksund_stensland_call(S_0=S_0, K=155, T=1, r=0.05, b=0, sigma=sigma)

# Print the results
print("First flat boundary (I_1):", round(I_1, 5))
print("Second flat boundary (I_2):", round(I_2, 5))
print("Time boundary (t_1):", str(round(t_1*365, 0)) + ' days')
print("h_1:", round(h_1, 5))
print("h_2:", round(h_2, 5))
print("Bjerksund-Stensland call approximation:", round(call_approx, 5))


First flat boundary (I_1): 451.54992
Second flat boundary (I_2): 460.66972
Time boundary (t_1): 226.0 days
h_1: 0.11135
h_2: 0.14164
Bjerksund-Stensland call approximation: 58.47639
